In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [2]:

def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data


In [3]:
indo_mgrs_grid_100km = gpd.read_file("/data/ksa/00_Data_Input/indo_mgrs_grid_100km.gpkg")
indo_mgrs_grid_100km.head()

,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,geometry
0,100000mE,800000mN,AP,46N,46NAP,"MULTIPOLYGON (((90.27865 8, 90.28351 7.22937, ..."
1,200000mE,800000mN,BP,46N,46NBP,"MULTIPOLYGON (((91.1854 8, 91.18862 7.23386, 9..."
2,300000mE,800000mN,CP,46N,46NCP,"MULTIPOLYGON (((92.09259 8, 92.09419 7.23656, ..."
3,400000mE,800000mN,DP,46N,46NDP,"MULTIPOLYGON (((93 8, 93 7.23747, 92.09419 7.2..."
4,500000mE,800000mN,EP,46N,46NEP,"MULTIPOLYGON (((93.90581 7.23656, 93 7.23747, ..."


In [4]:
df_indramayu = gpd.read_file("/data/ksa/00_Data_Input/grid_cent_25_SUKRA_4326_LBS.gpkg")
df_indramayu.head()

,gid,id_300_old,id_6km,nourut,s0_m2,s1_m2,s2_m2,s3_m2,sums0123,s0_proc,...,tahun,sumber,shape_leng,shape_area,prosen,layer,path,id,id_300,geometry
0,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,2019,BPS,0.010838,0.000007,60%,5grid300_jabar_luas_major_sawah_2017_s0s1s2s3_...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91243 -6.27003)
1,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,2019,BPS,0.010838,0.000007,60%,5grid300_jabar_luas_major_sawah_2017_s0s1s2s3_...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91261 -6.27003)
2,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,2019,BPS,0.010838,0.000007,60%,5grid300_jabar_luas_major_sawah_2017_s0s1s2s3_...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91279 -6.27003)
3,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,2019,BPS,0.010838,0.000007,60%,5grid300_jabar_luas_major_sawah_2017_s0s1s2s3_...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91297 -6.27003)
4,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,2019,BPS,0.010838,0.000007,60%,5grid300_jabar_luas_major_sawah_2017_s0s1s2s3_...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91315 -6.27003)


In [5]:
df_indramayu_mgrs = df_indramayu.sjoin(indo_mgrs_grid_100km, how="left", predicate='intersects')
df_indramayu_mgrs.head()

,gid,id_300_old,id_6km,nourut,s0_m2,s1_m2,s2_m2,s3_m2,sums0123,s0_proc,...,path,id,id_300,geometry,index_right,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS
0,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91243 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU
1,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91261 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU
2,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91279 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU
3,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91297 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU
4,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,22.180934,...,dbname='pgksa' host=10.0.36.90 port=5432 user=...,630901,3212416027,POINT (107.91315 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU


In [6]:
# df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# # df_ksa.iloc[:1000].to_csv("cek.csv")

# # df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
# df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
# df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

# df_ksa['index'] = [x.zfill(2) for x in df_ksa['index'].astype("str")]
# df_ksa['idpoint'] = df_ksa.idsegmen.astype('str') + df_ksa.idsubsegmen.astype('str') + '#' + df_ksa['index']

# df_ksa = df_ksa.query('idprov == "{0}"'.format(idprov))

# df_ksa.head()

In [7]:
df_indramayu_mgrs = df_indramayu_mgrs.reset_index()
df_indramayu_mgrs['lat'] = df_indramayu_mgrs.geometry.y
df_indramayu_mgrs['long'] = df_indramayu_mgrs.geometry.x
df_indramayu_mgrs.head(2)

,index,gid,id_300_old,id_6km,nourut,s0_m2,s1_m2,s2_m2,s3_m2,sums0123,...,id_300,geometry,index_right,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,lat,long
0,0,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,...,3212416027,POINT (107.91243 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU,-6.270031,107.912426
1,1,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,...,3212416027,POINT (107.91261 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU,-6.270031,107.912607


In [8]:
mgrs_not_done_ = df_indramayu_mgrs.MGRS.unique()
mgrs_not_done_

array(['48MZU', '48MZT'], dtype=object)

In [9]:
df_indramayu_mgrs.groupby('MGRS').MGRS.size()

MGRS
48MZT     5573
48MZU    82509
Name: MGRS, dtype: int64

In [10]:
df_indramayu_mgrs.head(2)

,index,gid,id_300_old,id_6km,nourut,s0_m2,s1_m2,s2_m2,s3_m2,sums0123,...,id_300,geometry,index_right,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,lat,long
0,0,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,...,3212416027,POINT (107.91243 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU,-6.270031,107.912426
1,1,267926,416027.0,1097,347,19962.847185,70037.183729,0.0,0.0,90000.030914,...,3212416027,POINT (107.91261 -6.27003),351,800000mE,9300000mN,ZU,48M,48MZU,-6.270031,107.912607


In [ ]:
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_indramayu_mgrs.loc[df_indramayu_mgrs.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    val = get_values(gc, bands_names, r['lat'], r['long'])
                    df_ksa_mgrs_tmp.loc[i,bands_names] = val
                    # print(r['lat'])
                    # break
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
                # break
                
            df_result = df_result[['index','MGRS','Sigma0_VH_db','periode']]
            with open('/data/ksa/03_Sampling/data/Indramayu/sampling_indramayu_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
            # break
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

48MZU available


 63%|██████▎   | 38/60 [1:57:24<1:09:24, 189.28s/it]

In [ ]:
1+5

In [ ]:
import concurrent.futures
import pandas as pd
from glob import glob
import pickle
from tqdm import tqdm

In [ ]:
# Function to process each MGRS
def process_mgrs(mgrs):
    df_result = pd.DataFrame()
    
    if check_image(mgrs):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/" + mgrs + "/"
            all_sentinel_data = glob(path_to_mgrs + "*.dim")
            df_ksa_mgrs = df_indramayu_mgrs.loc[df_indramayu_mgrs.MGRS == mgrs]
            
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset, gc, bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy().reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] = periode
                
                for i, r in df_ksa_mgrs_tmp.iterrows():
                    val = get_values(gc, bands_names, r['lat'], r['long'])
                    df_ksa_mgrs_tmp.loc[i, bands_names] = val
                
                df_result = pd.concat([df_result, df_ksa_mgrs_tmp])
            
            df_result = df_result[['index', 'MGRS', 'Sigma0_VH_db', 'periode']]
            output_path = f'/data/ksa/03_Sampling/data/Indramayu/sampling_indramayu_{mgrs}.pkl'
            with open(output_path, 'wb') as f:
                pickle.dump(df_result, f)
            
            print("Done sampling for : " + mgrs)
        
        except Exception as e: 
            print(f"Error on : {mgrs}, Error: {str(e)}")
    else:
        print(mgrs + " not available")

# Parallel processing of MGRS
def parallel_process_mgrs(mgrs_list):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(process_mgrs, mgrs_list)

# Run the parallel processing
parallel_process_mgrs(mgrs_not_done_)


In [ ]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

### Cek 

In [ ]:
## Cek VH Value Overtime on One Point

filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()

In [ ]:
## Open one Sampling Data

mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()

In [ ]:
df_.describe()